In [11]:
import numpy as np
import scipy.sparse
import math


''' Functions isPrime and nextPrime are taken form Geeks for Geeks '''
def isPrime(n):   
    if(n <= 1): 
        return False
    if(n <= 3): 
        return True 
    if(n % 2 == 0 or n % 3 == 0): 
        return False
      
    for i in range(5,int(math.sqrt(n) + 1), 6):  
        if(n % i == 0 or n % (i + 2) == 0): 
            return False
      
    return True
   
def nextPrime(N): 
    if (N <= 1): 
        return 2
  
    prime = N 
    found = False  
    while(not found): 
        prime = prime + 1
  
        if(isPrime(prime) == True): 
            found = True
  
    return prime 


def FSketch(X, k, p):
    ''''Documentation of FSKetch:
        Parameters:
           X: is a sparse array (numpy array or can be scipy sparse matrix) of shape (n, d),
              where n is number of samples and d is feature dimension
           k: reduced dimension value (must be postive integer)
            
           p: prime number next to the maximum element in X
        Returns:
          scipy sparse csr_matrix of shape (n, k)
    '''
    X = scipy.sparse.csr_matrix(X)
    d = X.shape[1]
    
    P = np.random.randint(0, p, d)
    
    random_map = np.array([], int)
    M = np.arange(k)
    for i in range(d//k):
        np.random.shuffle(M)
        random_map = np.concatenate((random_map, M))
    np.random.shuffle(M)
    random_map = np.concatenate((random_map, M[0:d%k]))
    np.random.shuffle(random_map)
    

    Temp = scipy.sparse.csr_matrix(X.multiply(P))
   
    new_X = np.zeros((X.shape[0], k))
    for i in range(k):
        index = np.where(random_map == i)[0]
        new_X[:,i] = Temp[:,index].sum(axis = 1).flatten() % p
    return scipy.sparse.csr_matrix(new_X)


def Hamming_distance(a,b):
    'Funtion to calculate hamming distance between array a and b'
    ham = 0
    for i in range(a.shape[1]):
        if a[:,i] != b[:,i]:
            ham += 1
    return ham

def FSketch_Ham_Estimate(a, b,  d, p ):
    ''''Documentation:
        Parameters:
          a: scipy sparse scr array
          b: sipy sparse scr  array 

          d: actual dimesion (dimension before reduction)

          p: prime number next to the maximum element in X
        Returns:
          integer values
    '''

    k = a.shape[1]
    ham = Hamming_distance(a,b)
    if ham/(k*(1 - 1/p)) < 1:
        return round(math.log(1 - ham/(k*(1 - 1/p))) / math.log(1 - 1/k))
    else:
        return ham

In [21]:
#Example
X = scipy.sparse.load_npz('Sample.npz')
print('Shape of actual matrix:', X.shape)

p = nextPrime(X.max())

new_X = FSketch(X, 1000, p)
print('Shape of compressed sktech matrix:', new_X.shape)

Shape of actual matrix: (100, 102660)
Shape of compressed sktech matrix: (100, 1000)


In [22]:
'''Hamming estimate'''
a = X[0,:]
b = X[1,:]
#hamming distance between a and b taken from actual matrix
print('Hamming distance between a and b is :', Hamming_distance(a,b))

a_new = new_X[0,:]
b_new = new_X[1,:]
# a_new and b_new corresponds to compressed sketch of a and b
d = X.shape[1]
print('Hamming estimate of a and b  using FSktech is :', FSketch_Ham_Estimate(a_new, b_new, d, p))

Hamming distance between a and b is : 225
Hamming estimate of a and b  using FSktech is : 226
